In [15]:
import os
import requests
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from datetime import datetime
from models.price_data_model import Base, PriceData
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

In [19]:

# Load environment variables
load_dotenv()

# Configuration
API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")
BASE_URL = 'https://www.alphavantage.co/query'
DATABASE_URL = os.getenv("DATABASE_URL")

# SQLAlchemy setup
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

def fetch_daily_prices(ticker):
    """Fetch daily adjusted stock prices and store them in the database."""
    params = {
        'function': 'TIME_SERIES_DAILY_ADJUSTED',
        'symbol': ticker,
        'outputsize': 'full',
        'apikey': API_KEY
    }

    response = requests.get(BASE_URL, params=params)
    if response.status_code != 200:
        raise Exception(f"API request failed: {response.status_code}")

    data = response.json().get('Time Series (Daily)', {})
    if not data:
        raise ValueError(f"No data returned for ticker {ticker}")

    records = []
    for date_str, values in data.items():
        record = PriceData(
            ticker=ticker,
            date=datetime.strptime(date_str, '%Y-%m-%d').date(),
            open=float(values['1. open']),
            high=float(values['2. high']),
            low=float(values['3. low']),
            close=float(values['4. close']),
            adjusted_close=float(values['5. adjusted close']),
            volume=int(values['6. volume']),
            source_api='Alpha Vantage',
            interval='1d',
            currency='USD',
            data_type='historical',
            created_at=datetime.utcnow()
        )
        records.append(record)

    session.bulk_save_objects(records)
    session.commit()
    print(f"Inserted {len(records)} price records for {ticker}")

if __name__ == '__main__':
    Base.metadata.create_all(engine)
    fetch_daily_prices('AAPL')  # Example


RuntimeError: 'cryptography' package is required for sha256_password or caching_sha2_password auth methods

In [3]:
ALTER USER 'root'@'localhost'
IDENTIFIED WITH mysql_native_password
BY '1234';

FLUSH PRIVILEGES;


SyntaxError: invalid syntax (479038977.py, line 1)